In [ ]:
! pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=ebe706c62ad0a7029613118b4a95e50fb04d765498bed116fea6bd1686719ef1
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
! pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("Datamanipulation").getOrCreate()

In [ ]:
spark

In [ ]:
# read our data - lives in a csv file

df = spark.read.option("header","true").csv("/Sample - EU Superstore.csv")

In [ ]:
# how many rows of the EU Superstore dataset have the country being France
df.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom|  North|OFF-ST-10000988|Office Supplies|     Storage|Fellowes 

In [ ]:
df.select("Country", when(df.Country == "France", 1).otherwise(0)).show()

+--------------+----------------------------------------------+
|       Country|CASE WHEN (Country = France) THEN 1 ELSE 0 END|
+--------------+----------------------------------------------+
|United Kingdom|                                             0|
|United Kingdom|                                             0|
|United Kingdom|                                             0|
|United Kingdom|                                             0|
|United Kingdom|                                             0|
|United Kingdom|                                             0|
|United Kingdom|                                             0|
|United Kingdom|                                             0|
|United Kingdom|                                             0|
|United Kingdom|                                             0|
|        France|                                             1|
|        France|                                             1|
|        France|                        

In [ ]:
# of those, how many are profitable?

df.select("Profit", when(df.Profit > 0, 1).otherwise(0)).show()

+-------+----------------------------------------+
| Profit|CASE WHEN (Profit > 0) THEN 1 ELSE 0 END|
+-------+----------------------------------------+
|   39.6|                                       1|
|      0|                                       0|
|  16.11|                                       1|
|   13.2|                                       1|
|  73.71|                                       1|
|  37.92|                                       1|
|  20.61|                                       1|
| 192.69|                                       1|
|1989.54|                                       1|
|   12.6|                                       1|
|   0.48|                                       0|
|760.305|                                       1|
|      0|                                       0|
|790.416|                                       1|
|-12.345|                                       0|
|  4.275|                                       1|
|      0|                      

In [ ]:
# how any different discount brackets exist? what are they?
grouped_df = df.groupBy("Discount").agg(count("Sales").alias("total"))
grouped_df.show()

+--------+-----+
|Discount|total|
+--------+-----+
|     0.3|   51|
|     0.7|    6|
|       0| 6134|
|     0.2|  125|
|    0.15|  407|
|    0.35|   45|
|     0.8|    4|
|    0.45|    2|
|     0.5| 1080|
|    0.65|   17|
|     0.6|  116|
|     0.1| 1737|
|    0.85|    2|
|     0.4|  274|
+--------+-----+



In [ ]:
# let's see the totl profit by discount bracket, make sure they are ordered by
a = df.groupBy("Discount").agg(sum("Profit").alias("total"))
a.orderBy(col("total").desc()).show()

+--------+-------------------+
|Discount|              total|
+--------+-------------------+
|       0| 383806.53000000026|
|     0.1|  126884.0309999999|
|    0.15| 24677.563499999975|
|     0.2| 2189.5499999999984|
|     0.8|           -460.284|
|     0.3| -758.4209999999999|
|    0.45|         -1103.1915|
|    0.85|          -3068.658|
|     0.7|          -5496.765|
|    0.65| -6221.965499999999|
|    0.35|          -9122.649|
|     0.6|-20517.456000000002|
|     0.4|-21346.427999999996|
|     0.5|         -96632.115|
+--------+-------------------+



In [ ]:
# who are the top 5 most profitable customers

df.orderBy(col("Profit").desc()).show()

+------+---------------+----------+----------+--------------+-----------+-------------------+-----------+---------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+-------+--------+--------+------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|      Customer Name|    Segment|           City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|  Sales|Quantity|Discount|Profit|
+------+---------------+----------+----------+--------------+-----------+-------------------+-----------+---------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+-------+--------+--------+------+
|  6073|ES-2017-1694278|19/08/2017|22/08/2017|   First Class|   JG-15805|         John Grady|  Corporate|         Berlin|              Berlin|       Germany|Central|OFF-ST-10000288|Office Supplies|     Sto

In [ ]:
# create a new column which is the value of the sale were there not discount applied. Hint: orginal = sales/(1-d)
df =   df.withColumn("no_profit", col("Sales") / (1 - col("Discount")))
df.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+------------------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|         no_profit|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+------------------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom|  Nor

In [ ]:
# create a temporary table for our superstore table in sql
df.registerTempTable("data")

In [ ]:
# use an SQL query to count the number of rows
spark.sql("Select count(*) from data").show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+



In [ ]:
# Use an SQL query to calculate the profit ratio for each country: hint, ratio is sum(profit)/sum(sales)
spark.sql("Select Country, sum(Profit)/sum(Sales) from data group by Country").show()

+--------------+--------------------------+
|       Country|(sum(Profit) / sum(Sales))|
+--------------+--------------------------+
|        Sweden|       -0.5745674280714466|
|       Germany|       0.17066792076621765|
|        France|       0.12693568221933804|
|       Belgium|       0.23508766583987942|
|       Finland|       0.18864296633316185|
|         Italy|       0.06844355185424991|
|        Norway|        0.2517747548521659|
|         Spain|       0.18941580658358978|
|       Denmark|       -0.4957190005664471|
|       Ireland|      -0.44426677493909256|
|   Switzerland|        0.2909201193350232|
|      Portugal|       -0.5761662270806188|
|       Austria|        0.2641908775042505|
|United Kingdom|       0.21170103540397134|
|   Netherlands|       -0.5298342790541865|
+--------------+--------------------------+



In [ ]:
# is the country with the largest profit ratio, the country with the largest profit?
spark.sql("Select Country, sum(Profit) from data group by Country").show()


+--------------+-------------------+
|       Country|        sum(Profit)|
+--------------+-------------------+
|        Sweden|-17519.366999999987|
|       Germany| 107322.82049999991|
|        France| 109029.00299999975|
|       Belgium|           11572.59|
|       Finland|            3905.73|
|         Italy| 19828.757999999965|
|        Norway|            5167.77|
|         Spain|  54390.11999999999|
|       Denmark|-4282.0470000000005|
|       Ireland| -7392.381000000003|
|   Switzerland|  7237.470000000001|
|      Portugal| -8703.059999999998|
|       Austria|           21442.26|
|United Kingdom| 111900.15000000001|
|   Netherlands| -41070.07499999996|
+--------------+-------------------+

